# Pip install

In [ ]:
pip install pymilvus grpcio-tools milvus

# *Restart Session before proceeding*

In [ ]:
from pymilvus import Collection, connections

# from milvus import Milvus, DataType

from urllib.request import urlretrieve
import tarfile
import numpy as np
import struct
import numpy as np
import requests
import time

In [ ]:
# Connect to Milvus server
TOKEN = '9ac70c2ae419bb92437b49afaee5e28a412b2b4a560366056e486c1e2b1f3af48f9a62a37551fa5735b5e4c27900abe963544dbf'
CLUSTER_ENDPOINT="https://in03-b29a7b5456b7b3f.api.gcp-us-west1.zillizcloud.com/"

connections.connect(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN)

collection = Collection('DeepDataset')
collection.load()

In [ ]:
# Test run

# # Number of vectors to search for at a time
# n = 100

# # Number of vecs to return for each search vector
# lim = 10

# # Example query vector
# query_vector = [0.5 for i in range(200)]

# # Specify search parameters
# search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

# # Run similarity search with specified search parameters
# results = collection.search(
# 	data=[query_vector for _ in range(n)],
# 	anns_field="vector",
# 	param=search_params,
# 	limit=lim,
# 	expr=None,
# 	consistency_level="Strong"
# )

In [ ]:
# # Process query results
# for i in range(len(results)):
#   print('Results for Search #', i)
#   for r in results[i]:
#     print(r)

# Define Experiment (Queries-Per-Second)

In [ ]:
def run_qps_test(n_vecs, n_runs=1):

	# Number of vecs to return for each search vector
	lim = 10

	vecs = []
	for i in range(n_vecs):
		query_vector = np.random.rand(96).tolist()
		vecs.append(query_vector)

	num_batches = n_vecs // 100

	# Specify search parameters
	search_params = {"metric_type": "IP", "params": {"nprobe": 10}}

	l_duration = []
	l_qps = []

	for run in range(n_runs):

		start_time = time.time()

		for i in range(0, num_batches, 100):
			batch = vecs[i:i+100]

			# Run similarity search with specified search parameters
			results = collection.search(
				data=batch,
				anns_field="vector",
				param=search_params,
				limit=lim,
				expr=None,
				consistency_level="Strong"
			)

		end_time = time.time()

		duration = end_time - start_time
		qps = n_vecs / duration

		l_duration.append(np.round(duration, 2))
		l_qps.append(qps)

	return l_duration, l_qps

def print_test(n_vecs, n_runs=1):
	duration, qps = run_qps_test(n_vecs, n_runs)
	print("Total duration:", duration, "secs")
	print("QPS: ", round(np.mean(qps), 2))

def remove_100k_rows(subset):
	subset *= 100000
	vector_ids_to_delete = [i+subset for i in range(100000)]
	collection.delete(expr=f'primary_key in {str(vector_ids_to_delete)}')

def remove_1M_rows(subset):
	subset *= 1000000
	vector_ids_to_delete = [i+subset for i in range(1000000)]
	collection.delete(expr=f'primary_key in {str(vector_ids_to_delete)}')

# Experiment Parameters

In [ ]:
n_vecs = 1000000
n_runs = 3

# Run Experiments

In [ ]:
print_test(n_vecs, n_runs)

Total duration: [65.8, 60.01, 59.99] secs
QPS:  16176.68


In [ ]:
ix = 0
remove_1M_rows(ix)
# remove_100k_rows(ix)

In [ ]:
print_test(n_vecs, n_runs)

Total duration: [52.0, 44.6, 42.99] secs
QPS:  21638.28


In [ ]:
ix = 1
remove_1M_rows(ix)
# remove_100k_rows(ix)

In [ ]:
print_test(n_vecs, n_runs)

Total duration: [40.43, 44.38, 40.02] secs
QPS:  24084.7


In [ ]:
ix = 2
remove_1M_rows(ix)
# remove_100k_rows(ix)

In [ ]:
print_test(n_vecs, n_runs)

Total duration: [40.01, 40.0, 42.8] secs
QPS:  24450.9


In [ ]:
ix = 3
remove_1M_rows(ix)
# remove_100k_rows(ix)

In [ ]:
print_test(n_vecs, n_runs)

Total duration: [39.96, 40.22, 40.19] secs
QPS:  24924.75


In [ ]:
# Clear out the last of the dataset

# ix = 4
# remove_1M_rows(ix)
# remove_100k_rows(ix)